# DS 1005 Hwk 3 - Belief Propagation

## Q1 - Sum-product algorithm 

In [1]:
import numpy as np 
import networkx as nx
from fglib import graphs, nodes, rv, inference, utils
from itertools import product

In [2]:
dist_f3 = [0.5, 0.5]
dist_f4 = [0.4,0.6]
px3x4=np.outer(dist_f3,dist_f4)
print(px3x4)
print(px3x4.shape)
px3x4=np.reshape(px3x4, np.shape(px3x4)+(1,))
print(px3x4)
print(px3x4.shape)
px2_conditioned_x3x4=[[[0.2,0.8],
                     [0.25,0.75],],
                     [[0.7,0.3],
                     [0.3,0.7]]]
print(px2_conditioned_x3x4)
dist_f234 =px3x4*px2_conditioned_x3x4
print(dist_f234)
print(dist_f234.shape)
px2= np.sum(dist_f234, axis=(0,1))
print(px2)
px1_conditioned_x2 = [[0.5,0.5],
                     [0.7,0.3]]
dist_f12 =px2[:,np.newaxis]*px1_conditioned_x2
print(dist_f12)
print(dist_f12.shape)

[[0.2 0.3]
 [0.2 0.3]]
(2, 2)
[[[0.2]
  [0.3]]

 [[0.2]
  [0.3]]]
(2, 2, 1)
[[[0.2, 0.8], [0.25, 0.75]], [[0.7, 0.3], [0.3, 0.7]]]
[[[0.04  0.16 ]
  [0.075 0.225]]

 [[0.14  0.06 ]
  [0.09  0.21 ]]]
(2, 2, 2)
[0.345 0.655]
[[0.1725 0.1725]
 [0.4585 0.1965]]
(2, 2)


In [3]:
def make_debug_graph():

    # Create factor graph
    fg = graphs.FactorGraph()

    # Create variable nodes
    x1 = nodes.VNode("x1", rv.Discrete)
    x2 = nodes.VNode("x2", rv.Discrete)
    x3 = nodes.VNode("x3", rv.Discrete)
    x4 = nodes.VNode("x4", rv.Discrete)

    # Create factor nodes
    f12 = nodes.FNode("f12")
    f234 = nodes.FNode("f234")
    f3 = nodes.FNode("f3")
    f4 = nodes.FNode("f4")

    # Add nodes to factor graph
    fg.set_nodes([x1, x2, x3, x4])
    fg.set_nodes([f12, f234, f3,f4 ])

    # Add edges to factor graph
    fg.set_edge(x1, f12)
    fg.set_edge(f12, x2)
    fg.set_edge(x2, f234)
    fg.set_edge(f234, x3)
    fg.set_edge(f234, x4)
    fg.set_edge(x3, f3)
    fg.set_edge(x4, f4)

    #add potential for f_3: p(x3)
    dist_f3 = [0.5, 0.5]
    f3.factor = rv.Discrete(dist_f3,x3)
    
    #add potential for f_4: p(x4)
    dist_f4 = [0.4,0.6]
    f4.factor = rv.Discrete(dist_f4,x4)
    
    # add potential for f_{234}: p(x2, x3, x4) = p(x2|x3,x4) p(x3,x4)
    px3x4=np.outer(dist_f3,dist_f4)
    px3x4=np.reshape(px3x4, np.shape(px3x4)+(1,))
    px2_conditioned_x3x4=[[[0.2,0.8],
                         [0.25,0.75],],
                         [[0.7,0.3],
                         [0.3,0.7]]]
    
    dist_f234 =px3x4*px2_conditioned_x3x4
    f234.factor = rv.Discrete(dist_f234,x3,x4,x2)
   
    # add potential for f_{12}:  p (x1,x2) = p(x1 | x2) p(x2)
    px1_conditioned_x2 = [[0.5,0.5],
                         [0.7,0.3]]
    px2= np.sum(dist_f234, axis=(0,1))
    dist_f12 =px2[:,np.newaxis]*px1_conditioned_x2
    f12.factor = rv.Discrete(dist_f12,x2,x1)
    # Perform sum-product algorithm on factor graph
    # and request belief of variable node x1
    
    belief = inference.sum_product(fg, x1)
    print(belief)
    return (fg)


### 1a. Implement sum-product algorithm 

In [4]:
def get_beliefs(model, n_iter=10):
    
    # initialize variable to factor messages 
    for v in model.get_vnodes():
        for f in v.neighbors():
            initial_msg = rv.Discrete(np.ones(2,), v)
            model[v][f]['object'].set_message(v, f, initial_msg)

    # run parallel updates for n_iter times 
    for i in range(n_iter):
        
        # update factor-to-variable messages
        for f in model.get_fnodes():
            for v in f.neighbors():
                msg = f.factor 
                for n in f.neighbors(exclusion=v):
                    msg *= model[n][f]['object'].get_message(n, f)
                for n in f.neighbors(exclusion=v):
                    msg = msg.marginalize(n, normalize=False)
                model[f][v]['object'].set_message(f, v, msg)
                
        # update variable-to-factor messages 
        for v in model.get_vnodes(): 
            for f in v.neighbors():
                msg = rv.Discrete(np.ones(2,), v)
                for n in v.neighbors(exclusion=f):
                    msg *= model[n][v]['object'].get_message(n, v)
                model[v][f]['object'].set_message(v, f, msg)

        # store beliefs of variable nodes 
        beliefs = {}
        for v in model.get_vnodes():
            beliefs[v] = v.belief(model)
    
    return beliefs

### 1b. Test on 4-node factor graph provided 

In [6]:
fg = make_debug_graph()
beliefs = get_beliefs(fg)
print("Belief of variable nodes ")
for (v,b) in beliefs.items():
    print(v, b)

[0.65897284 0.34102716]
Belief of variable nodes 
x1 [0.65897284 0.34102716]
x2 [0.20513578 0.79486422]
x3 [0.52640912 0.47359088]
x4 [0.28679718 0.71320282]


## Q2 - Low-Density Priority Check (LDPC)

In [8]:
from pyldpc import RegularH, CodingMatrixG # old version of API

### Part a:  

In [16]:
def get_factor_potentials(d_v): # where d_v = num of variable nodes in parity check equation 
    permutations = product([0,1], repeat=d_v)
    phi = np.zeros([2 for i in range(d_v)])
    for idx in permutations:
        if sum(idx) % 2 == 0: 
            phi[idx] = 1
        else:
            phi[idx] = 0
    return phi 

In [17]:
def make_ldpc_fg(H):
    """ Given an arbitrary parity check matrix create a corresponding factor graph """
    
    # initialize factor graph  
    fg = graphs.FactorGraph()
    
    # infer the number of factor and variable nodes required
    num_factors, num_variables = H.shape

    # create factor nodes 
    fnodes = [] 
    for i in range(num_factors):
        fnodes.append(nodes.FNode("f" + str(i)))
    
    # create variable nodes 
    vnodes = [] 
    for j in range(num_variables):
        vnodes.append(nodes.VNode("x" + str(j), rv.Discrete))
        
    # add nodes to factor graph 
    fg.set_nodes(vnodes)
    fg.set_nodes(fnodes)
    
    # add edges to factor graph based on H 
    for i in range(num_factors):
        for j in range(num_variables):
            if H[i,j] == 1:
                fg.set_edge(fnodes[i], vnodes[j])
                
    # set factor potentials 
    for fnode in fg.get_fnodes():
        vnodes2check = [vnode for vnode in fnode.neighbors()]
        fnode_potentials = get_factor_potentials(len(vnodes2check))
        fnode.factor = rv.Discrete(fnode_potentials, *vnodes2check)
        
    return fg

In [18]:
# create parity check matrix (note d_v = ones per column, d_c = ones per row)
H = RegularH(n=8, d_v=2, d_c=4)
print(H)
ldpc_fg = make_ldpc_fg(H)

[[1 1 1 1 0 0 0 0]
 [0 0 0 0 1 1 1 1]
 [0 0 1 1 0 1 1 0]
 [1 1 0 0 1 0 0 1]]


In [19]:
# define invalid codewords 
invalid_code = np.array([1, 0, 0, 0, 0, 0, 0, 0]) # should fail parity check 
print("Given H = {}, this code is expected to fail parity check: {}".format(H, invalid_code))

Given H = [[1 1 1 1 0 0 0 0]
 [0 0 0 0 1 1 1 1]
 [0 0 1 1 0 1 1 0]
 [1 1 0 0 1 0 0 1]], this code is expected to fail parity check: [1 0 0 0 0 0 0 0]


### Part b: 128-bit LDPC

In [20]:
H_b = RegularH(n=256, d_v=4, d_c=8)
print(H_b.shape)

(128, 256)


In [21]:
G_b = CodingMatrixG(H_b)
print(G_b.shape)

(131, 256)


In [22]:
def get_unary_potential(bit, err):
    phi = np.zeros(shape=(2,))
    phi[bit] = 1 - err 
    phi[1-bit] = err 
    return phi 

def set_unary_factors(fg, msg, err):
    for vnode in fg.get_vnodes():
        idx = int(str(vnode)[1:])
        phi = get_unary_potential(msg[idx-1], err)
        ufnode = nodes.FNode("u" + str(idx))
        ufnode.factor = rv.Discrete(phi, vnode)
        fg.set_node(ufnode)
        fg.set_edge(ufnode, vnode)
    return fg

In [23]:
msg = np.zeros(256, dtype=int)
fg_b = make_ldpc_fg(H_b)
fg_b = set_unary_factors(fg_b, msg, 0.05)
beliefs_b = get_beliefs(fg_b)

/anaconda3/envs/py36rocket/lib/python3.6/site-packages/fglib/rv.py:217: RuntimeWarning: overflow encountered in multiply
  pmf = self.pmf * other.pmf
/anaconda3/envs/py36rocket/lib/python3.6/site-packages/fglib/rv.py:292: RuntimeWarning: invalid value encountered in true_divide
  pmf = self.pmf / np.abs(np.sum(self.pmf))
/anaconda3/envs/py36rocket/lib/python3.6/site-packages/fglib/rv.py:217: RuntimeWarning: invalid value encountered in multiply
  pmf = self.pmf * other.pmf


In [24]:
for (v,b) in beliefs_b.items():
    print(v, b)

x0 [nan nan]
x1 [nan nan]
x2 [nan nan]
x3 [nan nan]
x4 [nan nan]
x5 [nan nan]
x6 [nan nan]
x7 [nan nan]
x8 [nan nan]
x9 [nan nan]
x10 [nan nan]
x11 [nan nan]
x12 [nan nan]
x13 [nan nan]
x14 [nan nan]
x15 [nan nan]
x16 [nan nan]
x17 [nan nan]
x18 [nan nan]
x19 [nan nan]
x20 [nan nan]
x21 [nan nan]
x22 [nan nan]
x23 [nan nan]
x24 [nan nan]
x25 [nan nan]
x26 [nan nan]
x27 [nan nan]
x28 [nan nan]
x29 [nan nan]
x30 [nan nan]
x31 [nan nan]
x32 [nan nan]
x33 [nan nan]
x34 [nan nan]
x35 [nan nan]
x36 [nan nan]
x37 [nan nan]
x38 [nan nan]
x39 [nan nan]
x40 [nan nan]
x41 [nan nan]
x42 [nan nan]
x43 [nan nan]
x44 [nan nan]
x45 [nan nan]
x46 [nan nan]
x47 [nan nan]
x48 [nan nan]
x49 [nan nan]
x50 [nan nan]
x51 [nan nan]
x52 [nan nan]
x53 [nan nan]
x54 [nan nan]
x55 [nan nan]
x56 [nan nan]
x57 [nan nan]
x58 [nan nan]
x59 [nan nan]
x60 [nan nan]
x61 [nan nan]
x62 [nan nan]
x63 [nan nan]
x64 [nan nan]
x65 [nan nan]
x66 [nan nan]
x67 [nan nan]
x68 [nan nan]
x69 [nan nan]
x70 [nan nan]
x71 [nan nan]
x7

## Q4 - Messsage passing on a tree

In [ ]:
from scipy.stats import norm

In [ ]:
norm.pdf(50, loc=50, scale=np.sqrt(10))

In [ ]:
norm.pdf(50, loc=60, scale=np.sqrt(10))

In [ ]:
norm.pdf(60, loc=50, scale=np.sqrt(10))

In [ ]:
norm.pdf(60, loc=60, scale=np.sqrt(10))